# Colab Setup  
> Make sure you configure notebook with GPU: Click Edit->notebook settings->hardware accelerator->GPU

> Uncomment the following cell after opening in Google colab. (Do not uncomment it in local setup.)  

<a target="_blank" href="https://colab.research.google.com/github/SEED-VT/FedDebug/blob/main/fault-localization/Reproduce_Table1-Table2.ipynb"> <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/> </a>


In [1]:
# !pip install pytorch-lightning
# !pip install diskcache
# !pip install dotmap
# !pip install torch torchvision torchaudio
# !pip install matplotlib
# !git clone https://github.com/SEED-VT/FedDebug.git
# # appending the path
# import sys
# sys.path.append("FedDebug/fault-localization/")

# Description

- It defines some variables for the simulation such as the learning rate, batch size, noise rate, number of clients, and number of epochs. 

- It then runs the simulation for the given configuration (`args`) related to Table 1 and Table 2 configurations. 

- Finally, it prints out the faulty client(s) localization accuracy, along with information about the distribution, number of faulty clients, total number of clients, architecture, and dataset used in the simulation.
 

In [2]:
import logging
import matplotlib.pyplot as plt
import time
from dotmap import DotMap
from pytorch_lightning import seed_everything
from torch.nn.init import kaiming_uniform_ 
from utils.faulty_client_localization.FaultyClientLocalization import FaultyClientLocalization
from utils.faulty_client_localization.InferenceGuidedInputs import InferenceGuidedInputs
from utils.FLSimulation import trainFLMain
from utils.fl_datasets import initializeTrainAndValidationDataset
from utils.util import aggToUpdateGlobalModel
from utils.util import testAccModel



logging.basicConfig(filename='example.log', level=logging.ERROR)
logger = logging.getLogger("pytorch_lightning")
seed_everything(786)



def evaluateFaultLocalization(predicted_faulty_clients_on_each_input, true_faulty_clients):
    true_faulty_clients = set(true_faulty_clients)
    detection_acc = 0
    for pred_faulty_clients in predicted_faulty_clients_on_each_input:
        print(f"+++ Faulty Clients {pred_faulty_clients}")
        correct_localize_faults = len(
            true_faulty_clients.intersection(pred_faulty_clients))
        acc = (correct_localize_faults/len(true_faulty_clients))*100
        detection_acc += acc
    fault_localization_acc = detection_acc / \
        len(predicted_faulty_clients_on_each_input)
    return fault_localization_acc


def runFaultyClientLocalization(client2models, exp2info, num_bugs, random_generator=kaiming_uniform_, apply_transform=True, k_gen_inputs=10, na_threshold=0.003, use_gpu=True):
    print(">  Running FaultyClientLocalization ..")
    input_shape = list(exp2info['data_config']['single_input_shape'])
    generate_inputs = InferenceGuidedInputs(client2models, input_shape, randomGenerator=random_generator, apply_transform=apply_transform,
                                            dname=exp2info['data_config']['name'], min_nclients_same_pred=5, k_gen_inputs=k_gen_inputs)
    selected_inputs, input_gen_time = generate_inputs.getInputs()

    start = time.time()
    faultyclientlocalization = FaultyClientLocalization(
        client2models, selected_inputs, use_gpu=use_gpu)

    potential_benign_clients_for_each_input = faultyclientlocalization.runFaultLocalization(
        na_threshold, num_bugs=num_bugs)
    fault_localization_time = time.time()-start
    return potential_benign_clients_for_each_input, input_gen_time, fault_localization_time



results = {}

# ====== Simulation ===== 

args = DotMap()
args.lr = 0.001
args.weight_decay = 0.0001
args.batch_size = 512

args.noise_rate = 1  # noise rate 0 to 1 
args.clients = 30 # keep under 30 clients and use Resnet18, Resnet34, or Densenet to evaluate on Colab 
args.epochs = 10  # range 10-25
args.faulty_clients_ids = "0" # can be multiple clients separated by comma e.g. "0,1,2"  but keep under args.clients clients and at max less than 7 


/home/gulzar/anaconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 786
Global seed set to 786


> Note: You can comment a complete cell to skip its execution in order to evalutate any particular configuration

 ### Table 1: resnet18, cifar10, iid distribution and 30 clients

In [3]:
args.model = "resnet18" # [resnet18, resnet34, resnet50, densenet121, vgg16]
args.dataset = "cifar10" # ['cifar10', 'femnist']
args.sampling = "iid" # [iid, "niid"] 
args.clients = 30 # keep under 30 clients and use Resnet18, Resnet34, or Densenet to evaluate on Colab 

# FL training
c2ms, exp2info = trainFLMain(args)
client2models = {k: v.model.eval() for k, v in c2ms.items()}


# Fault localazation
potential_faulty_clients, _, _ = runFaultyClientLocalization(
    client2models=client2models, exp2info=exp2info, num_bugs=len(exp2info['faulty_clients_ids']))
fault_acc = evaluateFaultLocalization(
    potential_faulty_clients, exp2info['faulty_clients_ids'])
# print(f"Fault Localization Acc: {fault_acc}")

print(f"#Fault Localization Accuracy: {fault_acc}, Distribution: {args.sampling},  Faulty clients: {len(args.faulty_clients_ids.split(','))}, Total Clients: {args.clients}, Architecture: {args.model}, Dataset: {args.dataset}")



  ***Simulating FL setup iid_resnet18_cifar10_clients_30_faulty_[0]_bsize_512_epochs_10_lr_0.001 ***
Files already downloaded and verified
Files already downloaded and verified
Spliting Datasets 50000 into parts:[1686, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666]
input shape, torch.Size([1, 3, 32, 32])
Training : .storage/checkpoints/iid_resnet18_cifar10_clients_30_faulty_[0]_bsize_512_epochs_10_lr_0.001/faulty_client_0_noise_rate_1_classes.ckpt


Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Train mod batch = 150, and drop_last = False


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 9: 100%|██████████| 4/4 [00:01<00:00,  2.48it/s, loss=2.31, train_acc=0.096, train_loss=2.300, val_acc=0.0928, val_loss=2.330] 

...

Epoch 00010: reducing learning rate of group 0 to 2.5000e-04.
Epoch 9: 100%|██████████| 4/4 [00:01<00:00,  2.69it/s, loss=0.624, train_acc=0.827, train_loss=0.503, val_acc=0.677, val_loss=1.520]
Training : .storage/checkpoints/iid_resnet18_cifar10_clients_30_faulty_[0]_bsize_512_epochs_10_lr_0.001/client_29.ckpt


Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Train mod batch = 130, and drop_last = False
Epoch 9: 100%|██████████| 4/4 [00:01<00:00,  2.45it/s, loss=0.678, train_acc=0.784, train_loss=0.569, val_acc=0.663, val_loss=1.330]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 4/4 [00:01<00:00,  2.44it/s, loss=0.678, train_acc=0.784, train_loss=0.569, val_acc=0.663, val_loss=1.330]
Total clients: 30
++Training is done: iid_resnet18_cifar10_clients_30_faulty_[0]_bsize_512_epochs_10_lr_0.001
>  Running FaultyClientLocalization ..
Same prediction threshold 5
+++ Faulty Clients {0}
+++ Faulty Clients {0}
+++ Faulty Clients {0}
+++ Faulty Clients {0}
+++ Faulty Clients {0}
+++ Faulty Clients {0}
+++ Faulty Clients {0}
+++ Faulty Clients {0}
+++ Faulty Clients {0}
+++ Faulty Clients {0}
#Fault Localization Accuracy: 100.0, Distribution: iid,  Faulty clients: 1, Total Clients: 30, Architecture: resnet18, Dataset: cifar10


 ### Table 1: densenet121, cifar10, niid distribution and 10 clients

In [4]:
args.model = "densenet121" # [resnet18, resnet34, resnet50, densenet121, vgg16]
args.dataset = "cifar10" # ['cifar10', 'femnist']
args.sampling = "niid" # [iid, "niid"] 
args.clients = 10


# FL training
c2ms, exp2info = trainFLMain(args)
client2models = {k: v.model.eval() for k, v in c2ms.items()}



# Fault localazation
potential_faulty_clients, _, _ = runFaultyClientLocalization(
    client2models=client2models, exp2info=exp2info, num_bugs=len(exp2info['faulty_clients_ids']))
fault_acc = evaluateFaultLocalization(
    potential_faulty_clients, exp2info['faulty_clients_ids'])
# print(f"Fault Localization Acc: {fault_acc}")

print(f"#Fault Localization Accuracy: {fault_acc}, Distribution: {args.sampling},  Faulty clients: {len(args.faulty_clients_ids.split(','))}, Total Clients: {args.clients}, Architecture: {args.model}, Dataset: {args.dataset}")



  ***Simulating FL setup niid_densenet121_cifar10_clients_10_faulty_[0]_bsize_512_epochs_10_lr_0.001 ***
Files already downloaded and verified
Files already downloaded and verified
Spliting Datasets 50000 into parts:[6237, 5701, 4138, 4019, 5802, 3981, 4709, 5651, 3541, 6221]
input shape, torch.Size([1, 3, 32, 32])
Training : .storage/checkpoints/niid_densenet121_cifar10_clients_10_faulty_[0]_bsize_512_epochs_10_lr_0.001/faulty_client_0_noise_rate_1_classes.ckpt


Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Train mod batch = 93, and drop_last = False
Epoch 9: 100%|██████████| 13/13 [00:04<00:00,  2.65it/s, loss=2.31, train_acc=0.151, train_loss=2.280, val_acc=0.101, val_loss=2.840]  

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 00010: reducing learning rate of group 0 to 2.5000e-04.
Epoch 9: 100%|██████████| 13/13 [00:04<00:00,  2.64it/s, loss=2.31, train_acc=0.151, train_loss=2.280, val_acc=0.101, val_loss=2.840]
Training : .storage/checkpoints/niid_densenet121_cifar10_clients_10_faulty_[0]_bsize_512_epochs_10_lr_0.001/client_1.ckpt


...

Train mod batch = 77, and drop_last = False


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 9: 100%|██████████| 13/13 [00:06<00:00,  1.93it/s, loss=0.322, train_acc=0.908, train_loss=0.303, val_acc=0.848, val_loss=0.857]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 13/13 [00:06<00:00,  1.93it/s, loss=0.322, train_acc=0.908, train_loss=0.303, val_acc=0.848, val_loss=0.857]
Total clients: 10
++Training is done: niid_densenet121_cifar10_clients_10_faulty_[0]_bsize_512_epochs_10_lr_0.001
>  Running FaultyClientLocalization ..
Same prediction threshold 5
+++ Faulty Clients {0}
+++ Faulty Clients {0}
+++ Faulty Clients {0}
+++ Faulty Clients {0}
+++ Faulty Clients {0}
+++ Faulty Clients {0}
+++ Faulty Clients {0}
+++ Faulty Clients {0}
+++ Faulty Clients {0}
+++ Faulty Clients {0}
#Fault Localization Accuracy: 100.0, Distribution: niid,  Faulty clients: 1, Total Clients: 10, Architecture: densenet121, Dataset: cifar10


### Table 2: Five Fautly Clients, densenet121, cifar10, and 30 clients 

In [5]:
args.sampling = "iid"
args.faulty_clients_ids = "0,1,3,4,7" # can be multiple clients separated by comma e.g. "0,1,2"  but keep under args.clients clients and at max less than 7 
args.model = "densenet121" # [resnet18, resnet34, resnet50, densenet121, vgg16]
args.dataset = "cifar10" # ['cifar10', 'femnist']
args.clients = 30 

# FL training
c2ms, exp2info = trainFLMain(args)
client2models = {k: v.model.eval() for k, v in c2ms.items()}

# Fault localazation
potential_faulty_clients, _, _ = runFaultyClientLocalization(
    client2models=client2models, exp2info=exp2info, num_bugs=len(exp2info['faulty_clients_ids']))
fault_acc = evaluateFaultLocalization(
    potential_faulty_clients, exp2info['faulty_clients_ids'])
# print(f"Fault Localization Acc: {fault_acc}")

print(f"#Table 2: Fault Localization Accuracy: {fault_acc}, Distribution: {args.sampling},  Faulty clients: {len(args.faulty_clients_ids.split(','))}, Total Clients: {args.clients}, Architecture: {args.model}, Dataset: {args.dataset}")



  ***Simulating FL setup iid_densenet121_cifar10_clients_30_faulty_[0, 1, 3, 4, 7]_bsize_512_epochs_10_lr_0.001 ***
Files already downloaded and verified
Files already downloaded and verified
Spliting Datasets 50000 into parts:[1686, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666, 1666]
input shape, torch.Size([1, 3, 32, 32])
Training : .storage/checkpoints/iid_densenet121_cifar10_clients_30_faulty_[0, 1, 3, 4, 7]_bsize_512_epochs_10_lr_0.001/faulty_client_0_noise_rate_1_classes.ckpt


Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Train mod batch = 150, and drop_last = False


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 9: 100%|██████████| 4/4 [00:02<00:00,  1.52it/s, loss=2.33, train_acc=0.125, train_loss=2.310, val_acc=0.0907, val_loss=2.500] 

...

Train mod batch = 130, and drop_last = False


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 9: 100%|██████████| 4/4 [00:02<00:00,  1.65it/s, loss=0.429, train_acc=0.885, train_loss=0.393, val_acc=0.739, val_loss=1.190]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 4/4 [00:02<00:00,  1.65it/s, loss=0.429, train_acc=0.885, train_loss=0.393, val_acc=0.739, val_loss=1.190]
Total clients: 30
++Training is done: iid_densenet121_cifar10_clients_30_faulty_[0, 1, 3, 4, 7]_bsize_512_epochs_10_lr_0.001
>  Running FaultyClientLocalization ..
Same prediction threshold 5
+++ Faulty Clients {0, 1, 3, 4, 7}
+++ Faulty Clients {0, 1, 3, 4, 7}
+++ Faulty Clients {0, 1, 3, 4, 7}
+++ Faulty Clients {0, 1, 3, 4, 7}
+++ Faulty Clients {0, 1, 3, 4, 7}
+++ Faulty Clients {0, 1, 3, 4, 7}
+++ Faulty Clients {0, 1, 3, 4, 7}
+++ Faulty Clients {0, 1, 3, 4, 7}
+++ Faulty Clients {0, 1, 3, 4, 7}
+++ Faulty Clients {0, 1, 3, 4, 7}
#Table 2: Fault Localization Accuracy: 100.0, Distribution: iid,  Faulty clients: 5, Total Clients: 30, Architecture: densenet121, Dataset: cifar10


### Table 2: Three Fautly Clients, resnet-50, cifar10, and 30 clients 

In [7]:
# args.sampling = "iid"
# args.faulty_clients_ids = "0,1,3" # can be multiple clients separated by comma e.g. "0,1,2"  but keep under args.clients clients and at max less than 7 
# args.model = "resnet50" # [resnet18, resnet34, resnet50, densenet121, vgg16]
# args.dataset = "cifar10" # ['cifar10', 'femnist']
# args.clients = 30 

# # FL training
# c2ms, exp2info = trainFLMain(args)
# client2models = {k: v.model.eval() for k, v in c2ms.items()}

# # Fault localazation
# potential_faulty_clients, _, _ = runFaultyClientLocalization(
#     client2models=client2models, exp2info=exp2info, num_bugs=len(exp2info['faulty_clients_ids']))
# fault_acc = evaluateFaultLocalization(
#     potential_faulty_clients, exp2info['faulty_clients_ids'])
# # print(f"Fault Localization Acc: {fault_acc}")

# print(f"#Table 2: Fault Localization Accuracy: {fault_acc}, Distribution: {args.sampling},  Faulty clients: {len(args.faulty_clients_ids.split(','))}, Total Clients: {args.clients}, Architecture: {args.model}, Dataset: {args.dataset}")